# PDMS v0.1 — CVSS-Style Scoring for Generative Protein Design (Interactive)
**How to use**
1. Upload `PDMS_template.csv` (same columns as provided).
2. Run the cells to compute Base, Temporal, Environmental, Final scores.
3. Get `PDMS_scored.csv` and a System×Layer heatmap.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

cat_map = {'low': 0.3, 'medium': 0.6, 'high': 0.9}
tec_map = {'unproven': 0.91, 'poc': 0.94, 'proof-of-concept': 0.94, 'functional': 0.97, 'active': 1.0, 'notdefined': 1.0, 'x': 1.0, 'not defined': 1.0}
rlv_map = {'official': 0.95, 'temporary': 0.96, 'workaround': 0.97, 'none': 1.0, 'notdefined': 1.0, 'not defined': 1.0, 'o': 0.95, 't': 0.96, 'w': 0.97, 'u': 1.0}
rfc_map = {'unknown': 0.92, 'reasonable': 0.96, 'confirmed': 1.0, 'notdefined': 1.0, 'not defined': 1.0, 'x': 1.0}
br_map  = {'low': 0.5, 'medium': 1.0, 'high': 1.5}
smm_map = {'advanced': 0.9, 'moderate': 0.95, 'minimal': 1.0}

def to_num(v, mapping):
    import pandas as pd
    if pd.isna(v): return None
    s = str(v).strip().lower()
    return mapping.get(s, mapping.get(s.replace('-',''), mapping.get(s.replace(' ','') , None)))

def invert(x):
    if x is None: return None
    return 1.0 - x

df = pd.read_csv('PDMS_template.csv')

for m in ["ACC","MUC","DKR","SCP","BHP","SOM","DET"]:
    df[m+"_num"] = df[m].apply(lambda x: to_num(x, cat_map))

df["TEC_num"] = df["TEC"].apply(lambda x: to_num(x, tec_map) if pd.notna(x) else 1.0).fillna(1.0)
df["RLV_num"] = df["RLV"].apply(lambda x: to_num(x, rlv_map) if pd.notna(x) else 1.0).fillna(1.0)
df["RFC_num"] = df["RFC"].apply(lambda x: to_num(x, rfc_map) if pd.notna(x) else 1.0).fillna(1.0)

df["BR_num"]  = df["BR"].apply(lambda x: to_num(x, br_map) if pd.notna(x) else 1.0).fillna(1.0)
df["SMM_num"] = df["SMM"].apply(lambda x: to_num(x, smm_map) if pd.notna(x) else 1.0).fillna(1.0)

df["E"] = df[["ACC_num","MUC_num","DKR_num"]].mean(axis=1)
df["E"] = (df["E"]*3 + invert(df["SCP_num"])) / 4

df["I"] = df[["BHP_num","SOM_num"]].mean(axis=1)
df["I"] = (df["I"]*2 + invert(df["DET_num"])) / 3

df["BaseScore"] = 10.0 * (0.6*df["I"] + 0.4*df["E"])

df["T_mult"] = df["TEC_num"] * df["RLV_num"] * df["RFC_num"]
df["ENV_mult"] = df["BR_num"] * df["SMM_num"]

df["FinalScore_raw"] = df["BaseScore"] * df["T_mult"] * df["ENV_mult"]
df["FinalScore"] = df["FinalScore_raw"].clip(upper=10.0)

def band(x):
    if x == 0: return "None"
    if x < 4.0: return "Low"
    if x < 7.0: return "Medium"
    if x < 9.0: return "High"
    return "Critical"
df["Severity"] = df["FinalScore"].apply(band)

def vector(row):
    return (
        f"PDMS:1.0/LAYER={row['Layer'].replace(' ','')}/"
        f"ACC={row['ACC'][0].upper()}/MUC={row['MUC'][0].upper()}/DKR={row['DKR'][0].upper()}/"
        f"SCP={row['SCP'][0].upper()}/BHP={row['BHP'][0].upper()}/SOM={row['SOM'][0].upper()}/DET={row['DET'][0].upper()};"
        f" T=TEC:{row['TEC']},RLV:{row['RLV']},RFC:{row['RFC']}; ENV=BR:{row['BR']},SMM:{row['SMM']}"
    )
df["Vector"] = df.apply(vector, axis=1)

df.to_csv('PDMS_scored.csv', index=False)
df.head(10)

In [ ]:
# Heatmap: System × Layer
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('PDMS_scored.csv')
pivot = df.pivot_table(index="System", columns="Layer", values="FinalScore", aggfunc="mean")

plt.figure(figsize=(7,5))
plt.imshow(pivot.values, aspect='auto')
plt.colorbar(label="Final Score (0–10)")
plt.xticks(range(pivot.shape[1]), pivot.columns, rotation=45, ha='right')
plt.yticks(range(pivot.shape[0]), pivot.index)
plt.title("PDMS Final Scores by System × Layer")
plt.tight_layout()
plt.show()